<a href="https://colab.research.google.com/github/KumbyYoz/KumbyYoz/blob/main/Chatbot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Installing the dependencies
!pip install google-cloud-aiplatform
!pip install pypdf
!pip install langchain
!pip install shapely==2.0.0 # provide a natural way to compute which features contribute to a prediction or contribute to the uncertainty of a prediction.
!pip install chromadb #open-source vector storage system (vector database) designed for the storing and retrieving vector embeddings

In [ ]:
from google.colab import auth

auth.authenticate_user()

In [ ]:
#!sudo add-apt-repository -y ppa:alessandro-strada/ppa 2>&1 > /dev/null
#!sudo apt-get update -qq 2>&1 > /dev/null
#!sudo apt -y install -qq google-drive-ocamlfuse 2>&1 > /dev/null
#!google-drive-ocamlfuse

In [ ]:
import os
from langchain.document_loaders import PyPDFLoader


def context_extractor(file_path):
    #text_extensions = ['.txt', '.csv']  # Add more text extensions if needed
    text_extensions = ['.csv']
    #pdf_extensions = ['.pdf']  # Add more PDF extensions if needed
    text = ''
    file_extension = os.path.splitext(file_path)[1].lower()

    if file_extension in text_extensions:
        with open(file_path, 'r') as file:
            text = file.read()
        return text
    elif file_extension in pdf_extensions:
        text = ''
        pdf_loader = PyPDFLoader(file_path)
        pages = pdf_loader.load_and_split()
        for i in pages:
            text = text +"\n\n" + i.page_content
        return text
    else:
        return 'Unknown file type'



In [ ]:
import os

def get_file_paths(directory):
    file_paths = []
    for root, _, files in os.walk(directory):
        for file in files:
            file_path = os.path.join(root, file)
            file_paths.append(file_path)
    return file_paths

In [ ]:
from langchain.text_splitter import CharacterTextSplitter

def create_context(dir_path):
  file_paths = get_file_paths(dir_path)
  texts_list = []
  for file_path in file_paths:
      print(file_path)
      text = context_extractor(file_path)
      text_splitter = CharacterTextSplitter(chunk_size=3000, chunk_overlap=100)
      context = text
      texts = text_splitter.split_text(context)
      for i in texts:
        texts_list.append(i)
  for chunk_index, chunk in enumerate(texts_list):
      # print(chunk)
      # Create a unique file name for each chunk
      chunk_file_name = f"/content/chunks/chunk_{chunk_index}.txt"

      # Save the chunk as a text file
      with open(chunk_file_name, 'w') as file:
          file.write(chunk)

      # Print the file path of the saved chunk
      print(f"Saved chunk {chunk_index} as {chunk_file_name}")

In [ ]:
def read_chunks(dir_path):
    context_paths = get_file_paths(dir_path)
    chunks = []
    for context_path in context_paths:
        with open(context_path, 'r') as file:
            chunk = file.read()
            chunks.append(chunk)
            print(chunk)
    return chunks

In [ ]:
dir_path = "/content/"

In [ ]:
file_path = '/content/FMCG_data_try2.csv'

In [ ]:
directory = "/content/"

In [ ]:
#read_chunks(/content/sample_data)

In [ ]:
def read_chunks(dir_path):
    context_paths = get_file_paths(dir_path)
    chunks = []
    for context_path in context_paths:
        with open(context_path, 'r') as file:
            chunk = file.read()
            chunks.append(chunk)
            print(chunk)
    return chunks

In [ ]:
import vertexai
import warnings
from langchain.embeddings import VertexAIEmbeddings
from langchain.llms import VertexAI
from langchain.vectorstores import Chroma
import os
warnings.filterwarnings("ignore")

PROJECT_ID = "studied-triode-407210"
#REGION = "southamerica-east1"
#REGION = "us-central1"
REGION = "northamerica-northeast1"

vertexai.init(project=PROJECT_ID, location=REGION)


In [ ]:
#ValueError: Unsupported region for Vertex AI, select from frozenset({'europe-west8', 'us-west3', 'europe-southwest1', 'europe-west4', 'asia-southeast1', 'asia-northeast2', 'australia-southeast1', 'europe-west2', 'us-east1', 'europe-west9', 'us-west4', 'europe-west3', 'us-east4', 'us-central1', 'europe-central2', 'asia-northeast3', 'europe-west1', 'asia-southeast2', 'asia-east1', 'northamerica-northeast2', 'southamerica-east1', 'northamerica-northeast1', 'australia-southeast2', 'europe-north1', 'us-west2', 'europe-west6', 'southamerica-west1', 'asia-south1', 'me-west1', 'us-south1', 'asia-east2', 'us-west1', 'asia-northeast1'})

In [ ]:
# parameters for model
#parameters = {
 #   "temperature": 0.2,
  #  "max_output_tokens": 380,
   # "top_p": 0.8,
    #"top_k": 40
#}

In [ ]:
# parameters for model
parameters = {
    "temperature": 0.1,
    "max_output_tokens": 1000,
    "top_p": 0.8,
    "top_k": 40
}

In [ ]:
#TextGenerationModel(model_id: str, endpoint_name: Optional[str] = None)

In [ ]:
#getting the text model
from vertexai.language_models import TextGenerationModel
model = TextGenerationModel.from_pretrained("text-bison@001")
vertex_embeddings = VertexAIEmbeddings(model_name="textembedding-gecko@001")

In [ ]:
# pass the path to directory containing documents
create_context('/content/documents')
#create_context('/content/sample_data/')

/content/documents/FMCG_data_try2.csv
Saved chunk 0 as /content/chunks/chunk_0.txt


In [ ]:
# load chunks
chunks = read_chunks('/content/chunks')
#chunks = read_chunks('/content/sample_data/')
#chunks = read_chunks()

# create embeddings from chunks
vector_index = Chroma.from_texts(chunks, vertex_embeddings).as_retriever()
#vector_index = Chroma.from_texts(chunks, vertex_embeddings).as_retriever(search_kwargs={"k": 4})

WarehouseID,WHManagerID,Locationtype,WHcapacitysize,zone,WHregionalzone,numrefillreql3m,transportissuel1y,Competitorinmkt,retailshopnum,whownertype,distributornum,floodimpacted,floodproof,electricsupply,distfromhub,workersnum,whestyear,storageissuereportedl3m,tempregmach,approvedwhgovtcertificate,whbreakdownl3m,govtcheckl3m,productwgton
WH100000,EID50000,Urban,Small,West,Zone 6,3,1,2,4651,Rented,24,0,1,1,91,29,,13,0,A,5,15,17115
WH100001,EID50001,Rural,Large,North,Zone 5,0,0,4,6217,Company Owned,47,0,0,1,210,31,,4,0,A,3,17,5074
WH100002,EID50002,Rural,Mid,South,Zone 2,1,0,4,4306,Company Owned,64,0,0,0,161,37,,17,0,A,6,22,23137
WH100003,EID50003,Rural,Mid,North,Zone 3,7,4,2,6000,Rented,50,0,0,0,103,21,,17,1,A,3,27,22115
WH100004,EID50004,Rural,Large,North,Zone 5,3,1,2,4740,Company Owned,42,1,0,1,112,25,2009,18,0,C,6,24,24071
WH100005,EID50005,Rural,Small,West,Zone 1,8,0,2,5053,Rented,37,0,0,1,152,35,2009,23,1,A,3,3,32134
WH100006,EID50006,Rural,Large,West,Zone 6,8,0,4,4449,Company Owned

In [ ]:
# main driver function
#This is the driver code, which will query the LLM and generate the responses
def run_query(question):
    # get the chunks having possibility of containing answer
    docs = vector_index.get_relevant_documents(question)
    answers = ''
    for i in docs:
        # query the LLM
        response = model.predict(
          f"Answer the question as precise as possible using the provided context.\
          Only take answers from the document provided\
          Do not get any answers from the internet sources\
           If the answer is not contained in the context, say 'answer not available \
           in context'. \n\n Context: \n {i.page_content} \nQuestion: \n {question} \n",
          **parameters
        )
        # append the answers
        answers = answers + "\n\n" + response.text

    final_response = model.predict(
        f"Given the extracted content and the question, create a conversational \
        final answer. If the answer is not contained in the context or if the context\
         is empty then, say 'answer not available in context'. \n\n Context: \n \
         {answers} \nQuestion: \n {question} \n",
        **parameters
        )
    return final_response.text

In [ ]:
print(run_query("WHregional_zone"))

The WHregional_zone is Zone 6.


In [ ]:
print(run_query(question = "what are the column names in this data"))

The column names are:

WarehouseID
WHManagerID
Locationtype
WHcapacitysize
zone
WHregionalzone
numrefillreql3m
transportissuel1y
Competitorinmkt
retailshopnum
whownertype
distributornum
floodimpacted
floodproof
electricsupply
distfromhub
workersnum
whestyear
storageissuereportedl3m
tempregmach
approvedwhgovtcertificate
whbreakdownl3m
govtcheckl3m
productwgton


In [ ]:
print(run_query(question = " Which WarehouseID has the heighest Competitorinmkt"))

WH100004


In [ ]:
print(run_query(question = " what is the Competitorinmkt for WarehouseID WH100011?"))

The answer is 2.


In [ ]:
print(run_query(question = "How many unique zones do we have?"))

There are 5 unique zones in the data.


In [ ]:
print(run_query(question = "what is the maximum in last column?"))

The maximum value in the last column is 7692.


In [ ]:
print(run_query(question = "what is the maximum distributor num"))

The maximum number of distributors is 28.


In [ ]:
print(run_query(question = "print top 5 rows of data"))

Sure, here are the top 5 rows of data:

WarehouseID,WHManagerID,Locationtype,WHcapacitysize,zone,WHregionalzone,numrefillreql3m,transportissuel1y,Competitorinmkt,retailshopnum,whownertype,distributornum,floodimpacted,floodproof,electricsupply,distfromhub,workersnum,whestyear,storageissuereportedl3m,tempregmach,approvedwhgovtcertificate,whbreakdownl3m,govtcheckl3m,productwgton
WH100000,EID50000,Urban,Small,West,Zone 6,3,1,2,4651,Rented,24,0,1,1,91,29,,13,0,A,5,15,17115
WH100001,EID50001,Rural,Large,North,Zone 5,0,0,4,6217,Company Owned,47,0,0,1,210,31,,4,0,A,3,17,5074
WH100002,EID50002,Rural,Mid,South,Zone 2,1,0,4,4306,Company Owned,64,0,0,0,161,37,,17,0,A,6,22,23137
WH100003,EID50003,Rural,Mid,North,Zone 3,7,4,2,6000,Rented,50,0,0,0,103,21,,17,1,A,3,27,22115
WH100004,EID50004,Rural,Large,North,Zone 5,3,1,2,4740,Company Owned,42,1,0,1,112,25,2009,18,0,C,6,24,24071


In [ ]:
print(run_query(question = "print a table of last 5 rows of data with headers"))

WarehouseID	WHManagerID	Locationtype	WHcapacitysize	zone	WHregionalzone	numrefillreql3m	transportissuel1y	Competitorinmkt	retailshopnum	whownertype	distributornum	floodimpacted	floodproof	electricsupply	distfromhub	workersnum	whestyear	storageissuereportedl3m	tempregmach	approvedwhgovtcertificate	whbreakdownl3m	govtcheckl3m	productwgton
WH100029	EID50029	Rural	Mid	South	Zone 3	1	0	4	5341	Company Owned	41	0	0	1	125	32		16	0	B	4	14	20115
WH100030	EID50030	Rural	Mid	North	Zone 3	1	0	2	5015	Company Owned	52	0	0	1	143	30		14	0	B	4	12	16100
WH100031	EID50031	Rural	Mid	West	Zone 3	1	0	1	4831	Company Owned	45	0	0	1	136	28		12	0	B	4	12	15100
WH100032	EID50032	Rural	Mid	South	Zone 3	1	0	1	4991	Company Owned	48	0	0	1	139	29		13	0	B	4	12	15600
WH100033	EID50033	Rural	Mid	North	Zone 3	1	0	1	4871	Company Owned	46	0	0	1	137	28		12	0	B	4	12	15300


In [ ]:
run_query(question = "print third column")

'The third column of the table is the location type. The location type can be urban, rural, or suburban.'

In [ ]:
print(run_query(question = "print a table of last 10 rows of data with headers"))

WarehouseID	WHManagerID	Locationtype	WHcapacitysize	zone	WHregionalzone	numrefillreql3m	transportissuel1y	Competitorinmkt	retailshopnum	whownertype	distributornum	floodimpacted	floodproof	electricsupply	distfromhub	workersnum	whestyear	storageissuereportedl3m	tempregmach	approvedwhgovtcertificate	whbreakdownl3m	govtcheckl3m	productwgton
WH100028	EID50028	Rural	Small	North	Zone 6	3	1	1	7692	Company Owned	62	1	0	0	154	20		0	0	NA	0	8	3123
WH100029	EID50029	Rural	Mid	South	Zone 3	1	0	4	5246	Company Owned	51	0	0	1	150	30		11	0	B	3	15	19113
WH100030	EID50030	Rural	Mid	South	Zone 3	6	0	2	5326	Company Owned	43	0	0	1	164	32	2013	16	0	B	5	19	20121
WH100031	EID50031	Rural	Mid	South	Zone 3	5	0	2	4547	Company Owned	41	0	0	1	119	34	2010	18	0	B	4	16	22106
WH100032	EID50032	Rural	Mid	South	Zone 3	1	0	2	4641	Company Owned	46	0	0	1	135	34	2013	18	0	B	4	19	23112
WH100033	EID50033	Rural	Mid	South	Zone 3	7	0	2	4891	Company Owned	42	0	0	1	124	34	2010	18	0	B	4	16	24114
WH100034	EID50034	Rural	Mid	South	Zone 

In [ ]:
questions = "how many WarehouseID?"

In [ ]:
model.predict("What is the meaning of life?")

I think the meaning of life is to find your passion and pursue it with all your heart. When you find something you love to do, it makes life so much more fulfilling. You feel like you're making a difference in the world, and you're happier and more productive.

In [ ]:
# The code is working above the next step is to pass a pdf or .csv and query from parsed documents.

In [ ]:
from google.api_core.client_options import ClientOptions

In [ ]:
!pip install google-cloud-documentai

In [ ]:
from google.api_core.client_options import ClientOptions
from google.cloud import documentai

In [ ]:
# Set endpoint to EU
options = ClientOptions(api_endpoint="eu-documentai.googleapis.com:443")
# Instantiates a client
client = documentai.DocumentProcessorServiceClient(client_options=options)

In [ ]:
#print(run_query(question=questions))

In [ ]:
questions = 'count WHregional_zone'

In [ ]:
print(run_query(question=questions))

In [ ]:
print(run_query(question=questions))

In [ ]:
model.predict("WarehouseID row 4")

In [ ]:
model.predict("zone for WarehouseID WH_100002")

In [ ]:
model.predict("What is WH_regional_zone for WarehouseID WH_100002")

In [ ]:
model.predict("What is the WHcapacity_size for WH_100019")